In [2]:
import numpy as np
import os
import glob
import tensorflow as tf
from slider import Beatmap



DATA_DIRECTORY = "beatmaps/"

themap = Beatmap.from_path('beatmaps/Bliitzit - Team Magma & Aqua Leader Battle Theme (Unofficial) (Sotarks) [Catastrophe].osu')
print(themap.stars())


6.413247155083175


In [34]:
def parse_hit_object(hit_object):
    object_type = type(hit_object).__name__
    time = hit_object.time.total_seconds()
    if object_type == "Circle":
        # [time posx posy]
        return [0,time,hit_object.position.x, hit_object.position.y,0,0,0,0,0]
    elif object_type == "Slider":
         # [hitobject_type time posx posy end_time num_points_on_curve length ticks tick_rate]
        return [1,time,hit_object.position.x,hit_object.position.y,hit_object.end_time.total_seconds(),len(hit_object.curve.points),hit_object.length,hit_object.ticks,hit_object.tick_rate]
    else:
        return [2,time,hit_object.position.x,hit_object.position.y,hit_object.end_time.total_seconds(),0,0,0,0]
       
def extend_data(training_data, max_objects):
    for datum in training_data:
        difference = len(datum[0]) - max_objects
        if difference < 0:
            for _ in range(0,abs(difference)):
                datum[0].append([0]*9)
    
def create_training_data():
    training_data = []
    file_names = [os.path.basename(x) for x in glob.glob("beatmaps/*.osu")]
    max_objects = 0
    for file_name in file_names:
        path = DATA_DIRECTORY+file_name
        beatmap = Beatmap.from_path(path)
        label = beatmap.stars()
        hit_objects = beatmap.hit_objects()
        if len(hit_objects) > max_objects:
            max_objects = len(hit_objects)
        hit_object_data = []
        for hit_object in hit_objects:
            hit_object_data.append(parse_hit_object(hit_object))
        training_data.append([hit_object_data,label])
    extend_data(training_data, max_objects)
    return training_data
        
training_data = create_training_data()


In [50]:
print(len(training_data))
print(len(training_data[0][0]))
X = []
y = []
for data in training_data:
    X.append(data[0])
    y.append(data[1])


X = np.array(X).reshape(-1, len(X),len(X[0]),1)
normalised_data = tf.keras.utils.normalize(X,axis=1)

39
2223
